In [6]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split


In [5]:
loc = '/home/cibin/Desktop/exl/TD/data/DS_v1/annotated/docanno_output/batch1/batch1_247/admin.jsonl'

df = pd.read_json(loc, lines=True)
df

,id,text,label,Comments
0,201,Agent: Thank you for calling Maple Bank. This ...,"[[329, 648, Authentication]]",[]
1,202,Agent: Good afternoon! This is Maple Bank Cust...,"[[344, 722, Authentication]]",[]
2,203,"Agent: Hello, this is Maple Bank’s Priority Su...","[[315, 710, Authentication]]",[]
3,204,"Agent: Thank you for calling Maple Bank, this ...","[[332, 750, Authentication]]",[]
4,205,Agent: Good afternoon! This is Maple Bank’s Se...,"[[293, 721, Authentication]]",[]
...,...,...,...,...
243,446,Agent: Thank you for calling Maple Bank's Merc...,"[[230, 477, Authentication]]",[]
244,447,"Agent: Maple Bank's Trust Services, this is La...","[[190, 362, Authentication]]",[]
245,448,Agent: Thank you for calling Maple Bank's Cust...,"[[214, 437, Authentication]]",[]
246,449,Agent: Maple Bank's Mortgage Servicing Departm...,"[[274, 456, Authentication]]",[]


In [9]:
# train test split
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape



((198, 4), (50, 4))

In [10]:
def is_token_label(token_indexes):
    
    """
    Determines if a token's position falls within a label's span in the text.
    """
    token_start_index, token_end_index, label_start_index,  label_end_index = token_indexes
    
    if label_start_index<= token_start_index and (label_end_index+2)>= token_end_index:
        
        return True
    return False


def add_label(token_df, label):
    """
    Adds a named entity label to tokens in the DataFrame that fall within the label's span.
    """
    
    label_start_index, label_end_index, label = label
    
    token_df['token_start_end_index_and_label_start_end_index'] = token_df.start_and_end_index.map(lambda x: x + ((label_start_index, label_end_index)))
    
    token_df['is_label'] = token_df.token_start_end_index_and_label_start_end_index.map(is_token_label)
    
    token_df.label[token_df.is_label]= label
    
    return token_df
    
def create_ner_text(text, labels):
    """
    Creates a DataFrame of tokens with NER labels from raw text and label spans.
    """
    
    tokens = text.split(' ')
    tokens_df = pd.DataFrame({"text":tokens})
    
    tokens_df['token_length'] = tokens_df.text.map(lambda x: len(x) + 1)
    tokens_df['end_index'] = tokens_df.token_length.cumsum()
    
    tokens_df['start_index'] = tokens_df.end_index - tokens_df.token_length
    
    tokens_df['label'] = 'O'
    
    tokens_df['start_and_end_index'] = list(zip(tokens_df.start_index, tokens_df.end_index))
    
    for label in labels:
        
        tokens_df = add_label(tokens_df, label)
        
    
    return tokens_df[['text','label']]

In [11]:
def preprocess_s1(base_loc, df):
    """
    Processes a DataFrame of transcripts and labels, converting each transcript into a tokenized
    NER-labeled format and saving as individual CSV files.
    """
    transcripts_list = df.text.tolist()
    labels_list =df.label.tolist()

    df_list = []
    for i, transcript in enumerate(transcripts_list):

        labels = labels_list[i]

        call_df = create_ner_text(transcript, labels)

        call_df.to_csv(os.path.join(base_loc, str(i)+'.csv'), index=None, sep=' ', header=None)

    

In [12]:
base_loc_train = '/home/cibin/Desktop/exl/TD/data/DS_v1/annotated/docanno_output/batch1/batch1_247/processed/train'
base_loc_test = '/home/cibin/Desktop/exl/TD/data/DS_v1/annotated/docanno_output/batch1/batch1_247/processed/test'


In [14]:
preprocess_s1(base_loc=base_loc_train, df=train_df)

/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning

/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning

/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning

In [16]:
preprocess_s1(base_loc=base_loc_test, df=test_df)

/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_df.label[token_df.is_label]= label
/tmp/ipykernel_53523/3704455144.py:33: SettingWithCopyWarning

In [19]:
def preprocess_s2(base_loc_input, output_loc):
    """
    Concatenates multiple text files from an input directory into a single output file,
    preserving the original content structure with newline separation between files.
    """

    files_list = os.listdir(base_loc_input)
    complete_text = ''
    for file in files_list:

        with open(os.path.join(base_loc_input, file), 'r') as f:
            data = f.read()

        complete_text = complete_text + data +'\n'

    complete_text = complete_text.strip()

    with open(output_loc, 'w+') as f:
        f.write(complete_text)

In [20]:
preprocess_s2(base_loc_input=base_loc_train, output_loc='/home/cibin/Desktop/exl/TD/data/DS_v1/annotated/docanno_output/batch1/batch1_247/processed/final/train.txt')

In [22]:
preprocess_s2(base_loc_input=base_loc_test, output_loc='/home/cibin/Desktop/exl/TD/data/DS_v1/annotated/docanno_output/batch1/batch1_247/processed/final/test.txt')